In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|██████████| 5/5 [02:50<00:00, 34.20s/it]


In [9]:
deals[0].details

"  That's the best deal we've ever seen on this model. A 1-year Allstate warranty applies. Buy Now at eBay        "

In [10]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [11]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [12]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Refurb Unlocked Apple iPhone 14 Pro 256GB Phone for $360 + free shipping
Details: That's the best deal we've ever seen on this model. A 1-year Allstate warranty applies. Buy Now at eBay
Features: 
URL: https://www.dealnews.com/Refurb-Unlocked-Apple-iPhone-14-Pro-256-GB-Phone-for-360-free-shipping/21791870.html?iref=rss-c142

Title: Refurb Apple iPhone 13 128GB Smart

In [13]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [14]:
result = get_recommendations()

In [15]:
len(result.deals)

5

In [17]:
print(result.deals[1])

product_description='The bObsweep UltraVision Self-Empty Robot Vacuum & Mop is a high-performance cleaning device that simultaneously vacuums and mops your floors. With a generous 6L dust collector, it offers up to 1200W suction power to tackle dirt and debris efficiently. Its advanced mapping technology allows it to navigate your home seamlessly. Enjoy features like odor filtering and a user-friendly mobile app for easy operation.' price=260.0 url='https://www.dealnews.com/products/bObsweep/bObsweep-Ultra-Vision-Self-Empty-Robot-Vacuum-Mop/487591.html?iref=rss-f1912'


In [18]:
from agents.scanner_agent import ScannerAgent

In [19]:
agent = ScannerAgent()
result = agent.scan()

In [20]:
result

DealSelection(deals=[Deal(product_description='The Unlocked Motorola Moto G Power 5G is equipped with 128GB of storage, allowing you to store all your important files, photos, and apps. The device features a vibrant display that enhances your viewing experience, complemented by powerful battery life to keep you connected throughout the day. This phone is perfect for users looking for a reliable and affordable smartphone option. It is now available for $129.99.', price=129.99, url='https://www.dealnews.com/Android-Phones-at-Best-Buy-Up-to-400-off-free-shipping/21791884.html?iref=rss-c142'), Deal(product_description='The Samsung S84F Series 65" OLED 4K UHD Vision AI TV provides stunning picture quality with deep blacks and vivid colors, enhancing your movie and gaming experience. It is designed for ultimate immersion with advanced technology that adjusts brightness and contrast for exceptional clarity. This sleek and modern television fits seamlessly into any living space. It is offered 